In [1]:
"""
Extracts pages from wikipedia and saves the content as text to ./data/sources

Requires wikipedia see https://github.com/goldsmith/Wikipedia
Install with
> pip install wikipedia

"""

import wikipedia

wikipedia.set_lang("fr")

page_title = "Racing Club de Strasbourg Alsace"

pg = wikipedia.page(page_title)

# save the content
output_file = page_title.replace(" ", "_").replace("'", "").lower() + ".txt"

with open(f"{output_file}", "w", encoding="utf-8") as f:
    f.write(pg.content)

print(f"content saved in ./data/sources/wikipedia/{output_file}")

content saved in ./data/sources/wikipedia/racing_club_de_strasbourg_alsace.txt


## Partie chunk

In [2]:
import glob
import typing as t
import uuid
import pandas as pd
import tiktoken

def chunkit(input_: t.List[str], window_size: int = 3, overlap: int = 1) -> t.List[str]:
    assert (
        overlap < window_size
    ), f"overlap {overlap} needs to be smaller than window size {window_size}"
    start_ = 0
    chunks = []
    while start_ + window_size < len(input_):
        chunks.append(input_[start_ : start_ + window_size])
        start_ = start_ + window_size - overlap
    # add the remaining paragraphs
    if start_ < len(input_):
        chunks.append(input_[start_ - len(input_) :])

    extracts = ["\n".join(chk) for chk in chunks]

    return extracts

In [3]:
# chargez les noms des fichiers txt

source_path = "*.txt"
source_files = glob.glob(source_path)

In [4]:
allchunks = []
for filename in source_files:
        print(f"-- loading {filename}")
        with open(filename, "r", encoding="utf-8") as f:
            txt = f.read()

        # split the text over line returns
        lines = txt.split("\n")
        # remove empty lines
        lines = [par.strip() for par in lines if len(par.strip()) > 1]
        chunked_version = chunkit(lines)

        allchunks += chunked_version

        print(f"extracted {len(chunked_version)} allchunks")

    # set as dataframe, to make it easier to add other info for each chunk and save it to json later on
data = pd.DataFrame(data=allchunks, columns=["text"])

    # create unique id for each chunk
data["uuid"] = [str(uuid.uuid4()) for i in range(len(data))]

    # count the number of tokens
print("-- count tokens")
encoding = tiktoken.get_encoding("cl100k_base")

data["token_count"] = data.text.apply(lambda txt: len(encoding.encode(txt)))

# check the max number of tokens in the dataset
print(f"max number of tokens: {max(data.token_count)}")
print(f"distribution of number of tokens: {data.token_count.describe()}")

# save to json
output_file_json = "eu_20240303.json"
with open(output_file_json, "w", encoding="utf-8") as f:
    data.to_json(f, force_ascii=False, orient="records", indent=4)

print(f"-- saved to {output_file_json}")

-- loading association_sportive_de_monaco_football_club.txt
extracted 89 allchunks
-- loading football_club_de_nantes.txt
extracted 108 allchunks
-- loading olympique_de_marseille.txt
extracted 187 allchunks
-- loading olympique_gymnaste_club_de_nice.txt
extracted 83 allchunks
-- loading olympique_lyonnais.txt
extracted 151 allchunks
-- loading paris_saint-germain_football_club.txt
extracted 153 allchunks
-- loading racing_club_de_lens.txt
extracted 139 allchunks
-- loading racing_club_de_strasbourg_alsace.txt
extracted 99 allchunks
-- loading stade_brestois_29.txt
extracted 76 allchunks
-- loading toulouse_football_club.txt
extracted 69 allchunks
-- count tokens
max number of tokens: 1901
distribution of number of tokens: count    1154.000000
mean      313.941941
std       264.289296
min        13.000000
25%       116.000000
50%       247.000000
75%       436.000000
max      1901.000000
Name: token_count, dtype: float64
-- saved to eu_20240303.json


## Partie Create Collection + weaviate

In [5]:
# sauvegarde clef d'acces Weaviate
import os
os.environ['OPENAI_API_KEY'] = ""
os.environ["WEAVIATE_KEY"] = ""
os.environ["WEAVIATE_CLUSTER_URL"] = ""

In [6]:
# verifier que la clé a bien été enregistrée
assert os.environ.get('OPENAI_API_KEY')  is not None
assert os.environ.get('WEAVIATE_KEY')  is not None
assert os.environ.get('WEAVIATE_CLUSTER_URL') is not None

In [7]:

import weaviate

def connect_to_weaviate() -> weaviate.client.WeaviateClient:
    client = weaviate.connect_to_wcs(
        cluster_url=os.environ["WEAVIATE_CLUSTER_URL"],
        auth_credentials=weaviate.AuthApiKey(os.environ["WEAVIATE_KEY"]),
        headers={
            "X-OpenAI-Api-Key": os.environ["OPENAI_API_KEY"],
        },
    )
    # check that the vector store is up and running
    if client.is_live() & client.is_ready() & client.is_connected():
        print(f"client is live, ready and connected ")

    assert (
        client.is_live() & client.is_ready()
    ), "Weaviate client is not live or not ready or not connected"
    return client



In [8]:
client = connect_to_weaviate()

C:\Users\yetsu\Documents\Lib\site-packages\weaviate\__init__.py:128: DeprecationWarning: Dep010: Importing AuthApiKey from weaviate is deprecated. Please import it from its specific module: weaviate.auth
  _Warnings.root_module_import(name, map_[name])


client is live, ready and connected 


In [9]:
"""
"""
# import weaviate.classes as wvc
from weaviate.classes.config import Property, DataType, Configure, Reconfigure

In [10]:
# collection_name must start with an Uppercase
collection_name = "Yamine_football_fr_2024"
assert collection_name.capitalize() == collection_name

In [11]:
# create schema
properties = [
    Property(
        name="uuid",
        data_type=DataType.UUID,
        skip_vectorization=True,
        vectorize_property_name=False,
    ),
    Property(
        name="text",
        data_type=DataType.TEXT,
        skip_vectorization=False,
        vectorize_property_name=False,
    ),
]

In [12]:
# set vectorizer
vectorizer = Configure.Vectorizer.text2vec_openai(
    vectorize_collection_name=False, model="text-embedding-3-small"
)

In [13]:
# create collection
# 1st check if collection does not exist
all_existing_collections = client.collections.list_all().keys()
collection_exists = collection_name in all_existing_collections
# assert not collection_exists, f"{collection_name} (exists {collection_exists})"

In [14]:
all_existing_collections

dict_keys(['AIActKnowledgeBase', 'AIAct_240218', 'AIAct_240219', 'AIAct_240220', 'Alexis_union_mars_2024_002', 'AlignmentKnowledgeBase', 'Clement_20240311', 'Mathis_sta_full', 'Etienne_union_mars_2024_001', 'Irene_ww2', 'Etienne_afr_mars_2024_001', 'Mathis_sta_2024_002', 'New_union_mars_2024_11', 'New_union_mars_2024_12', 'Clement_20240312', 'Vincent_recettes_mars_2024_002', 'Vincent_recettes_mars_2024_003', 'Yamine_football_fr_2024', 'Mathis_sta_complet', 'Ludivine_progres_mars_2024', 'Ludivine_progres_mars_2024_4', 'Irene_ww22', 'Union_europeenne', 'Sadjia_akziz_mars_2024_001'])

In [15]:
# alternatively you can choose to delete the collection and all its records with:
if collection_exists:
    client.collections.delete(collection_name)
    print(f"collection {collection_name} has been deleted")

# now create the collection
collection = client.collections.create(
        name=collection_name,
        vectorizer_config=vectorizer,
        properties=properties,
    )
# add stopwords
print("add French stopwords")
import nltk

nltk.download("stopwords")
from nltk.corpus import stopwords

collection.config.update(
    # Note, use Reconfigure here (not Configure)
    inverted_index_config=Reconfigure.inverted_index(
        stopwords_additions=list(stopwords.words("french"))
        )
    )

# check collection has been created
all_existing_collections = client.collections.list_all().keys()

if collection_name in all_existing_collections:
    print(f"{collection_name} has been created")
else:
    print(f"{collection_name} has NOT been created")

client.close()

collection Yamine_football_fr_2024 has been deleted
add French stopwords
Yamine_football_fr_2024 has been created


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yetsu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Partie Embed

In [16]:
import pandas as pd

In [17]:
collection_name = "Yamine_football_fr_2024"

input_file = "eu_20240303.json"
data = pd.read_json(input_file)
data = data[["uuid", "text"]]
print("-- loaded ", data.shape[0], "items")

-- loaded  1154 items


In [18]:
client = connect_to_weaviate()

C:\Users\yetsu\Documents\Lib\site-packages\weaviate\__init__.py:128: DeprecationWarning: Dep010: Importing AuthApiKey from weaviate is deprecated. Please import it from its specific module: weaviate.auth
  _Warnings.root_module_import(name, map_[name])


client is live, ready and connected 


In [19]:
# load the collection
collections = client.collections.get(collection_name)

In [20]:
# insert the data
batch_result = collections.data.insert_many(data.to_dict(orient="records"))

if batch_result.has_errors:
    print(batch_result.errors)
    raise RuntimeError("stopping")

# finaly verify that the data has been inserted
# reload the collection
collection = client.collections.get(collection_name)

records_num = collection.aggregate.over_all(total_count=True).total_count
print(f"collection {collection_name} now has {records_num} records")

collection Yamine_football_fr_2024 now has 1154 records


# vectorisation des questions
## Partie Run Retriever

In [21]:
def search(query, search_mode = "hybrid"):
    metadata = ["distance", "certainty", "score", "explain_score"]
    if search_mode == "hybrid":
        response = collection.query.hybrid(
            query=query,
            query_properties=["text"],
            limit=2,
            return_metadata=metadata,
        )
    elif search_mode == "near_text":
        response = collection.query.near_text(
            query=query,
            limit=2,
            return_metadata=metadata,
        )
    elif search_mode == "bm25":
        response = collection.query.bm25(
            query=query,
            limit=2,
            return_metadata=metadata,
        )
    return response


In [22]:
for item in response.objects:
    print("--" * 20)
    print(item.properties.get("uuid"))
    print()
    print(item.properties.get("text"))
    print()
    print(f"metadata: {item.metadata.__dict__}")
    print()


NameError: name 'response' is not defined

## Partie Génération: Question Reponse

In [ ]:
!pip install -r requirements.txt

In [23]:
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.chains import SequentialChain

# Local
from prompts import Prompt


In [24]:
class Generate(object):
    def __init__(self,
            model: str = "gpt-3.5-turbo-0125",
            temperature: float = 0.9
        ) -> None:

        llm = ChatOpenAI(model=model, temperature=temperature)

        context_chain = LLMChain(
            llm=llm,
            prompt=Prompt.prompt_generate_groundtruth,
            output_key="answer",
            verbose=False,
        )
        self.overall_context_chain = SequentialChain(
            chains=[context_chain],
            input_variables=["context"],
            output_variables=["answer"],
            verbose=True,
        )

    def generate_question_answer(self, context):
        response = self.overall_context_chain({"context": context})
        return response["answer"]

In [25]:
import pandas as pd
data = pd.read_json('eu_20240303.json')
data.shape

(1154, 3)

In [26]:
import json
qa = []
bad_formatted = []
for i, chunk in data.iterrows():

    gen = Generate()
    answer = gen.generate_question_answer(chunk.text)
    print(i, answer)
    try:
        qa.append(json.loads(answer))
    except:
        bad_formatted.append(answer)

C:\Users\yetsu\Documents\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...

> Finished chain.
0 {
    "question": "Quelle est l'année de fondation de l'AS Monaco FC?",
    "reponse": "L'AS Monaco FC a été fondé en 1924."
}


> Entering new SequentialChain chain...

> Finished chain.
1 {
    "question": "Quels sont les joueurs de l'AS Monaco FC qui ont été champions du monde avec l'équipe de France ?",
    "reponse": "Emmanuel Petit, Lilian Thuram, David Trezeguet, Thierry Henry, Kylian Mbappé, Fabien Barthez, Djibril Sidibé et Thomas Lemar."
}


> Entering new SequentialChain chain...

> Finished chain.
2 {
    "question": "Quel club a fourni des joueurs à l'équipe de France depuis les années 1950?",
    "reponse": "L'AS Monaco FC"
}


> Entering new SequentialChain chain...

> Finished chain.
3 {
    "question": "Quand l'AS Monaco a-t-elle été fondée?",
    "reponse": "L'AS Monaco a été fondée en 1924 par fusion de quatre clubs de Monaco et de Beausoleil."
}


> Entering new SequentialChain chain...

> Finished chain

KeyboardInterrupt: 

In [27]:
qa_df = pd.DataFrame(qa)

In [28]:
output_file_json = "qa_football_fr_20240312.json"
with open(output_file_json, "w", encoding="utf-8") as f:
    qa_df.to_json(f, force_ascii=False, orient="records", indent=4)


## Partie RAG

In [29]:
class Retrieve(object):
    collection_name = "Yamine_football_fr_2024"

    def __init__(self, query: str, search_params: t.Dict) -> None:
        self.client = connect_to_weaviate()
        assert self.client is not None
        assert self.client.is_live()

        # retrieval
        self.collection = self.client.collections.get(Retrieve.collection_name)
        self.query = query
        self.search_mode = search_params.get("search_mode")
        self.response_count = search_params.get("response_count")

        # output
        self.response = ""
        self.chunk_texts = []
        self.metadata = []

    # retrieve
    def search(self):
        metadata = ["distance", "certainty", "score", "explain_score"]
        if self.search_mode == "hybrid":
            self.response = self.collection.query.hybrid(
                query=self.query,
                # query_properties=["text"],
                limit=self.response_count,
                return_metadata=metadata,
            )
        elif self.search_mode == "near_text":
            self.response = self.collection.query.near_text(
                query=self.query,
                limit=self.response_count,
                return_metadata=metadata,
            )
        elif self.search_mode == "bm25":
            self.response = self.collection.query.bm25(
                query=self.query,
                limit=self.response_count,
                return_metadata=metadata,
            )

    def get_context(self):
        texts = []
        metadata = []
        if len(self.response.objects) > 0:
            for i in range(min([self.response_count, len(self.response.objects)])):
                prop = self.response.objects[i].properties
                texts.append(f"--- \n{prop.get('text')}")
                metadata.append(self.response.objects[i].metadata)
            self.chunk_texts = texts
            self.metadata = metadata

    def close(self):
        self.client.close()

    def process(self):
        self.search()
        self.get_context()
        self.close()

In [30]:
params = {
        "search_mode": "hybrid",
        "response_count": 3,
        "model": "gpt-3.5-turbo-0125",
        "temperature": 0.5,
    }

query = "Quels sont les joueurs internationaux jeunes qui n'ont jamais été appelés en équipe senior ?"

In [31]:
from langchain.prompts import ChatPromptTemplate


prompt_generative_context = ChatPromptTemplate.from_template(
        """"Prends le rôle d'un consultant sportif spécialisé dans les clubs de football français.
Vous êtes invité à donner votre analyse sur la performance récente des clubs de Ligue 1 et à discuter de leur histoire, de leurs points forts et de leurs défis actuels.

En tant qu'IA, tu peux utiliser tes connaissances générales pour répondre à la question mais surtout n'invente rien.

Indique clairement
- Si le contexte ne permet pas de répondre à la question
- Si tes connaissances générales ne te permettent pas de répondre à la question

Voici une question et un contexte.
Réponds à la question en prenant compte l'information dans le contexte.
Écris une réponse dans un style concis .

--- Le contexte:
{context}
--- La question:
{query}
Ta réponse:
"""
    )


In [32]:
ret = Retrieve(query, params)
ret.process()

C:\Users\yetsu\Documents\Lib\site-packages\weaviate\__init__.py:128: DeprecationWarning: Dep010: Importing AuthApiKey from weaviate is deprecated. Please import it from its specific module: weaviate.auth
  _Warnings.root_module_import(name, map_[name])


client is live, ready and connected 


In [33]:
class Generate(object):
    def __init__(self, model: str = "gpt-3.5-turbo-0125", temperature: float = 0.5) -> None:
        self.model = model
        self.temperature = temperature
        llm = ChatOpenAI(model=model, temperature=temperature)

        llm_chain = LLMChain(
            llm=llm,
            prompt=prompt_generative_context,
            output_key="answer",
            verbose=True,
        )

        self.overall_context_chain = SequentialChain(
            chains=[llm_chain],
            input_variables=["context", "query"],
            output_variables=["answer"],
            verbose=True,
        )
        # outputs
        self.answer = ""

    def generate_answer(self, chunk_texts: t.List[str], query: str) -> str:
        response_context = self.overall_context_chain(
            {"context": "\n".join(chunk_texts), "query": query}
        )
        self.answer = response_context["answer"]

In [34]:
print(query)
gen = Generate()
gen.generate_answer(ret.chunk_texts, query)

Quels sont les joueurs internationaux jeunes qui n'ont jamais été appelés en équipe senior ?


> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: "Prends le rôle d'un consultant sportif spécialisé dans les clubs de football français.
Vous êtes invité à donner votre analyse sur la performance récente des clubs de Ligue 1 et à discuter de leur histoire, de leurs points forts et de leurs défis actuels.

En tant qu'IA, tu peux utiliser tes connaissances générales pour répondre à la question mais surtout n'invente rien.

Indique clairement
- Si le contexte ne permet pas de répondre à la question
- Si tes connaissances générales ne te permettent pas de répondre à la question

Voici une question et un contexte.
Réponds à la question en prenant compte l'information dans le contexte.
Écris une réponse dans un style concis .

--- Le contexte:
--- 
En grisé, les sélections de joueurs internationaux chez les jeunes mais n'ayant jamais été app

In [35]:
gen.answer

"Désolé, je ne peux pas répondre à cette question car le contexte ne fournit pas d'information spécifique sur les joueurs internationaux jeunes qui n'ont jamais été appelés en équipe senior."

In [37]:
query2 = "Quel est le parcours européen mémorable de l'AS Monaco FC lors de la saison 2003-2004?"
ret2 = Retrieve(query2, params)
ret2.process()
gen.generate_answer(ret2.chunk_texts, query2)

C:\Users\yetsu\Documents\Lib\site-packages\weaviate\__init__.py:128: DeprecationWarning: Dep010: Importing AuthApiKey from weaviate is deprecated. Please import it from its specific module: weaviate.auth
  _Warnings.root_module_import(name, map_[name])


client is live, ready and connected 


> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: "Prends le rôle d'un consultant sportif spécialisé dans les clubs de football français.
Vous êtes invité à donner votre analyse sur la performance récente des clubs de Ligue 1 et à discuter de leur histoire, de leurs points forts et de leurs défis actuels.

En tant qu'IA, tu peux utiliser tes connaissances générales pour répondre à la question mais surtout n'invente rien.

Indique clairement
- Si le contexte ne permet pas de répondre à la question
- Si tes connaissances générales ne te permettent pas de répondre à la question

Voici une question et un contexte.
Réponds à la question en prenant compte l'information dans le contexte.
Écris une réponse dans un style concis .

--- Le contexte:
--- 
La saison 2003-2004 est une saison mémorable pour le club. C'est l'année du Périple Rouge et Blanc. Qualifié pour la Ligue des champions, l'AS Monaco 

In [ ]:
gen.answer

In [40]:
ret2.chunk_texts

["--- \nLa saison 2003-2004 est une saison mémorable pour le club. C'est l'année du Périple Rouge et Blanc. Qualifié pour la Ligue des champions, l'AS Monaco FC s'illustre dès le premier tour en battant historiquement le Deportivo La Corogne, en s'imposant 8 buts à 3 et battant ainsi un nouveau record en nombre de buts marqués sur un match. Monaco compte alors des joueurs tels que le gardien Flavio Roma ou encore Jérôme Rothen, Patrice Évra, Hugo Ibarra, mais surtout sa paire offensive magique, l'ailier droit Ludovic Giuly et l'attaquant espagnol prêté par le Real Madrid, Fernando Morientes. En huitième de finale, Monaco élimine le Lokomotiv Moscou, puis hérite du Real Madrid en quart de finale.\nLe club espagnol est battu à la suite de deux matchs spectaculaires de la part de Ludovic Giuly et de Fernando Morientes notamment. Lors de la demi-finale, l'AS Monaco FC élimine une nouvelle fois une grosse écurie, le Chelsea Football Club. Cependant, les joueurs monégasques ne rééditeront pa